In [1]:
# 匯入所有要用的Package
import os
import pymongo
import xlwings as xw
import pandas as pd
import gc



In [2]:
# 連接MongoClient、Excel表，以及抓取CMoney的更新函數
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['Data']
wb = xw.Book(r'C:\Users\ChastLai\Documents\Data\CMoney一堆測試 備份20220912.xlsm')
func_ = wb.macro("CM更新")

In [4]:
# 預設在所有表單更新，股票代號頁面除外
sheets_name = [s.name for s in wb.sheets]
for stk_number in sheets_name:
    if stk_number == "股票代號":
        continue
    # 選定sheets
    sheets = wb.sheets[stk_number]
    # 執行CMoney更新
    func_(stk_number)
    # 將股票名稱也儲存起來
    stk_name = wb.sheets[stk_number].range("A5").value.split(' ')[1]
    # 計算更新完的總row數量
    row = sheets.range("B5").current_region.last_cell.row
    # 將還原開高低收改表頭名稱，與真實開高低收分開
    sheets.range("AT5:AW5").value = ['還原開盤價', '還原最高價', '還原最低價', '還原收盤價']
    # 將sheet內容轉換為DataFrame
    df = sheets.range("B5:BS" + str(row)).options(pd.DataFrame, header=1).value
    # 將需要的值都放進DataFrame，方便後續匯入DataFrame
    df['股票代號'] = stk_number
    df['股票名稱'] = stk_name
    df['日期'] = df.index
    # 有許多空值，基本上都用前值補齊
    df = df.fillna(method='ffill')
    # 如果要重置db內容，開啟此項
    # df[stk_number].delete_many({})
    if db[stk_number].count_documents({}) != len(df):
        db[stk_number].insert_many(df.to_dict('records'))
    del df
    gc.collect()
    sheets.delete()


In [3]:
sheets = wb.sheets[0]
row = sheets.range("A5").current_region.last_cell.row
stk_map = dict(zip(wb.sheets[0].range("A6:A" + str(row)).value, wb.sheets[0].range("B6:B" + str(row)).value))

In [32]:
for i in db.list_collection_names():
    # 修改錯誤的股票名稱
    db[i].update_many(filter={'股票名稱': '台積電'}, update={"$set": {'股票名稱': stk_map[i]}}, upsert=False)

In [9]:
import datetime
import time
# 預設在所有表單更新，股票代號頁面除外
while True:
    if datetime.datetime.now().hour == 8:
        wb = xw.Book(r'C:\Users\ChastLai\Documents\Data\CMoney一堆測試 備份20220912.xlsm')
        sheets_name = [s.name for s in wb.sheets]

        #執行CMoney更新
        func_ = wb.macro("CM全部更新")
        func_()

        for stk_number in sheets_name:
            if stk_number == "股票代號":
                continue
            # 選定sheets
            sheets = wb.sheets[stk_number]

            # 將股票名稱也儲存起來
            stk_name = wb.sheets[stk_number].range("A5").value.split(' ')[1]
            # 計算更新完的總row數量
            row = sheets.range("B5").current_region.last_cell.row
            # 將還原開高低收改表頭名稱，與真實開高低收分開
            sheets.range("AT5:AW5").value = ['還原開盤價', '還原最高價', '還原最低價', '還原收盤價']
            # 將sheet內容轉換為DataFrame
            df = sheets.range("B5:BS" + str(row)).options(pd.DataFrame, header=1).value
            # 將需要的值都放進DataFrame，方便後續匯入DataFrame
            df['股票代號'] = stk_number
            df['股票名稱'] = stk_name
            df['日期'] = df.index
            # 有許多空值，基本上都用前值補齊
            df = df.fillna(method='ffill')
            # update資料
            for i in df.to_dict('records'):
                db[stk_number].update_one(filter={"日期": i["日期"]}, update={"$set": i}, upsert=True)

            del df
            gc.collect()
        wb.save()
        wb.close()
        break
    else:
        time.sleep(360)


In [4]:
sheets_name = [s.name for s in wb.sheets]
for stk_number in sheets_name:
    if stk_number == "股票代號":
        continue
    # 選定sheets
    sheets = wb.sheets[stk_number]
    try:
        sheets.range("A1").value = sheets.range("A1").value.replace(r'/10/', r'/2/')
    except Exception as e:
        print(e)
        pass

In [7]:
import datetime
datetime.datetime.now().hour

17

# 單日更新報表

In [1]:
import datetime
import time
import os
import pymongo
import xlwings as xw
import pandas as pd
import gc
# 連接MongoClient、Excel表，以及抓取CMoney的更新函數
while True:
    if datetime.datetime.now().hour == 8:

        client = pymongo.MongoClient('mongodb://localhost:27017/')
        db = client['Index']
        wb = xw.Book("CMoney指數.xlsm")
        func_ = wb.macro("CM更新")
        func_('VIX')
        func_('TWA00')

        sheets = wb.sheets['VIX']
        row = sheets.range("B5").current_region.last_cell.row
        # 將sheet內容轉換為DataFrame
        df = sheets.range("B5:I" + str(row)).options(pd.DataFrame, header=1, index=0).value
        df.dropna(inplace=True)
        db['VIX'].update_many(filter={'日期': df.loc[df.index[0], '日期']}, update={'$set': df.to_dict('records')[0]}, upsert=True)

        sheets = wb.sheets['TWA00']
        row = sheets.range("B5").current_region.last_cell.row
        # 將sheet內容轉換為DataFrame
        df = sheets.range("B5:AJ" + str(row)).options(pd.DataFrame, header=1, index=0).value
        df.dropna(inplace=True)
        db['TWA00'].update_one(filter={'日期': df.loc[df.index[0], '日期']}, update={'$set': df.to_dict('records')[0]}, upsert=True)

        # 換表單
        db = client['Data']
        wb = xw.Book("CMoney單日更新.xlsm")
        sheets = wb.sheets['工作表2']
        # 這邊注意是要更新哪天資料
        sheets.range("C2:BT2").value = '20220923' #datetime.datetime.today().strftime('%Y%m%d')
        func_ = wb.macro("CM更新")
        func_('工作表2')
        row = sheets.range("A5").current_region.last_cell.row
        # 將還原開高低收改表頭名稱，與真實開高低收分開
        sheets.range("AU5:AX5").value = ['還原開盤價', '還原最高價', '還原最低價', '還原收盤價']
        # 將sheet內容轉換為DataFrame
        df = sheets.range("A5:BT" + str(row)).options(pd.DataFrame, header=1, index=0).value
        df.columns = df.columns.str.replace('.\d', '', regex=True)
        for i in db.list_collection_names():
            data = df[df['股票代號'] == i]
            try:
                db[i].update_one(filter={'日期': data.loc[data.index[0], '日期']}, update={'$set': data.to_dict('records')[0]}, upsert=True)
            except Exception as e:
                print(f'error in {i} because {e}')
    break
            #time.sleep(3500)


IndexError: index 0 is out of bounds for axis 0 with size 0

# 特別新增指數

In [2]:
import datetime
import time
import os
import pymongo
import xlwings as xw
import pandas as pd
import gc
# 連接MongoClient、Excel表，以及抓取CMoney的更新函數

client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['Index']
wb = xw.Book("CMoney指數.xlsm")
func_ = wb.macro("CM更新")
func_('VIX')
func_('TWA00')
sheets = wb.sheets['VIX']
row = sheets.range("B5").current_region.last_cell.row
# 將sheet內容轉換為DataFrame
df = sheets.range("B5:I" + str(row)).options(pd.DataFrame, header=1, index=0).value
df.dropna(inplace=True, how='all')
#db['VIX'].insert_many(df.to_dict('records'))
db['VIX'].update_one(filter={'日期': df.loc[df.index[0], '日期']}, update={'$set': df.to_dict('records')[0]}, upsert=True)

sheets = wb.sheets['TWA00']
row = sheets.range("B5").current_region.last_cell.row
sheets.range("C5:F5").value = ['還原開盤價', '還原最高價', '還原最低價', '還原收盤價']
# 將sheet內容轉換為DataFrame
df = sheets.range("B5:AJ" + str(row)).options(pd.DataFrame, header=1, index=0).value
df.dropna(inplace=True, how='all')
# db['TWA00'].insert_many(df.to_dict('records'))
db['TWA00'].update_one(filter={'日期': df.loc[df.index[0], '日期']}, update={'$set': df.to_dict('records')[0]}, upsert=True)

# # 換表單
db = client['Data']
wb = xw.Book("CMoney單日更新.xlsm")
sheets = wb.sheets['工作表2']
# 這邊注意是要更新哪天資料
sheets.range("C2:BT2").value = '20220927' #datetime.datetime.today().strftime('%Y%m%d')
func_ = wb.macro("CM更新")
func_('工作表2')
row = sheets.range("A5").current_region.last_cell.row
# 將還原開高低收改表頭名稱，與真實開高低收分開
sheets.range("AU5:AX5").value = ['還原開盤價', '還原最高價', '還原最低價', '還原收盤價']
# 將sheet內容轉換為DataFrame
df = sheets.range("A5:BT" + str(row)).options(pd.DataFrame, header=1, index=0).value
df.columns = df.columns.str.replace('.\d', '', regex=True)
for i in db.list_collection_names():
    data = df[df['股票代號'] == i]
    try:
        db[i].update_one(filter={'日期': data.loc[data.index[0], '日期']}, update={'$set': data.to_dict('records')[0]}, upsert=True)
    except Exception as e:
        print(f'error in {i} because {e}')